In [2]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

In [3]:
data=pd.read_csv("general_data.csv")

In [4]:
data.head(5)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [5]:
data.drop(columns=['EmployeeCount','EmployeeID'],inplace=True)

In [7]:
data.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64

In [8]:
data.mean()

Age                           36.923810
DistanceFromHome               9.192517
Education                      2.912925
JobLevel                       2.063946
MonthlyIncome              65029.312925
NumCompaniesWorked             2.694830
PercentSalaryHike             15.209524
StandardHours                  8.000000
StockOptionLevel               0.793878
TotalWorkingYears             11.279936
TrainingTimesLastYear          2.799320
YearsAtCompany                 7.008163
YearsSinceLastPromotion        2.187755
YearsWithCurrManager           4.123129
dtype: float64

In [9]:
new_Num_Comp_worked =np.where(data["NumCompaniesWorked"].isnull(),3,data["NumCompaniesWorked"])
new_Total_Wrk_Yrs =np.where(data["TotalWorkingYears"].isnull(),11,data["TotalWorkingYears"])
data["NumCompaniesWorked"]=new_Num_Comp_worked
data["TotalWorkingYears"]=new_Total_Wrk_Yrs

In [11]:
objList = data.select_dtypes(include = "object").columns

In [13]:
label_encoder = preprocessing.LabelEncoder()
attrition_encoded = data.apply(label_encoder.fit_transform)

In [14]:
rf_attmodel = RandomForestClassifier(n_estimators=1000,max_features =2,oob_score=True)

In [15]:
features =["Age","BusinessTravel", "Department", "DistanceFromHome", "Education", "EducationField", "Gender", 
           "JobLevel", "JobRole",  "MaritalStatus", "MonthlyIncome", "NumCompaniesWorked", "Over18",
           "PercentSalaryHike", "StandardHours", "StockOptionLevel",  "TotalWorkingYears", "TrainingTimesLastYear",
           "YearsAtCompany","YearsSinceLastPromotion", "YearsWithCurrManager"]

In [16]:
rf_attmodel.fit(X=attrition_encoded[features],y=attrition_encoded["Attrition"])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [17]:
print("OOB Accuracy :",rf_attmodel.oob_score_)

OOB Accuracy : 1.0


In [18]:
for feature, imp in zip(features,rf_attmodel.feature_importances_):
    print(feature,imp)

Age 0.09636315152461333
BusinessTravel 0.028633782062022016
Department 0.02614765463594904
DistanceFromHome 0.0701100722159107
Education 0.04138192535436899
EducationField 0.04238365442862454
Gender 0.018034631961394088
JobLevel 0.03760500671579278
JobRole 0.05637841241286433
MaritalStatus 0.03954318027815959
MonthlyIncome 0.09253978488355977
NumCompaniesWorked 0.05524596662810284
Over18 0.0
PercentSalaryHike 0.06532785128971157
StandardHours 0.0
StockOptionLevel 0.03414721046089666
TotalWorkingYears 0.08613107578567943
TrainingTimesLastYear 0.045181806790847684
YearsAtCompany 0.06784853905439715
YearsSinceLastPromotion 0.04270712587516811
YearsWithCurrManager 0.05428916764193744


In [19]:
tree_model =tree.DecisionTreeClassifier(max_depth = 8)

In [21]:
IndVar_Attr =pd.DataFrame([attrition_encoded['Age'],
                           attrition_encoded['MonthlyIncome'],
                           attrition_encoded['TotalWorkingYears'],
                           attrition_encoded['DistanceFromHome'],
                           attrition_encoded['YearsAtCompany'],
                           attrition_encoded['PercentSalaryHike'],
                           attrition_encoded['NumCompaniesWorked'],
                           attrition_encoded['JobRole'],
                           attrition_encoded['YearsWithCurrManager']]).T

In [22]:
tree_model.fit(X=IndVar_Attr,y=attrition_encoded["Attrition"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [23]:
with open("Attrition_Model.dot",'w') as f:
    f=tree.export_graphviz(tree_model,feature_names=["Age","MonthlyIncome","TotalWorkingYears","DistanceFromHome","YearsAtCompany","PercentSalaryHike","NumCompaniesWorked","JobRole","YearsWithCurrManager"],out_file=f);

In [24]:
print("Classification Accuracy with the features  is :", tree_model.score(X=IndVar_Attr,y=attrition_encoded["Attrition"]))

Classification Accuracy with the features  is : 0.9018140589569161
